In [1]:
# Display the current working directory
getwd();
# If necessary, change the path below to the directory where the data files are stored.
# "." means current directory. On Windows use a forward slash / instead of the usual \.
workingDir = "lung";
setwd(workingDir);
# Load WGCNA package
library(WGCNA)
allowWGCNAThreads()
# The following setting is important, do not omit.
options(stringsAsFactors = FALSE);

[1] "/home/jovyan/work/phd/cancers"

Loading required package: dynamicTreeCut

Loading required package: fastcluster


Attaching package: ‘fastcluster’


The following object is masked from ‘package:stats’:

    hclust





Attaching package: ‘WGCNA’


The following object is masked from ‘package:stats’:

    cor




Allowing multi-threading with up to 2 threads.


In [2]:
add_legend <- function(...) {
  opar <- par(fig=c(0, 1, 0, 1), oma=c(0, 0, 0, 0), 
    mar=c(0, 0, 0, 0), new=TRUE)
  on.exit(par(opar))
  plot(0, 0, type='n', bty='n', xaxt='n', yaxt='n')
  legend(...)
}

In [3]:
datExpr <- t(read.csv("mainTable.csv", check.names=FALSE, row.names = "0"))
genes <- dimnames(datExpr)[[2]]
samples <- dimnames(datExpr)[[1]]
datExpr <- as.data.frame(apply(datExpr,2, as.numeric)) #numeric
dimnames(datExpr)[[1]] <- samples
allTraits <- read.csv("files.dat")

In [4]:
datExpr[1:5,1:5]

,ENSG00000000938,ENSG00000001084,ENSG00000001631,ENSG00000002933,ENSG00000003147
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
dc0ce32f-d5aa-489f-ba4b-e01443815121.FPKM.txt.gz,4.24,11.76,4.21,10.27,8.74
2b33014e-c7ff-41aa-82bf-3121756639ad.FPKM.txt.gz,2.53,85.19,12.01,4.88,1.71
397e302e-393e-4f54-b360-fcffe394271e.FPKM.txt.gz,1.90,40.94,2.98,7.64,13.39
037b5a94-487e-4f5b-90a3-d696eb4a8a3e.FPKM.txt.gz,4.10,0.96,6.13,15.01,5.76
5e010a9a-7f65-434b-ae4c-43948461187c.FPKM.txt.gz,20.25,3.06,4.05,14.71,3.18


In [6]:
log2_1 <- function(x){
    log2(x+1)
}
datExpr <- as.data.frame(apply(datExpr,2, log2_1))
dimnames(datExpr)[[1]] <- samples

In [7]:
datExpr[1:5,1:5]

,ENSG00000000938,ENSG00000001084,ENSG00000001631,ENSG00000002933,ENSG00000003147
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
dc0ce32f-d5aa-489f-ba4b-e01443815121.FPKM.txt.gz,2.389567,3.6735564,2.381283,3.494416,3.283922
2b33014e-c7ff-41aa-82bf-3121756639ad.FPKM.txt.gz,1.819668,6.4294486,3.701549,2.555816,1.438293
397e302e-393e-4f54-b360-fcffe394271e.FPKM.txt.gz,1.536053,5.3902550,1.992768,3.111031,3.846995
037b5a94-487e-4f5b-90a3-d696eb4a8a3e.FPKM.txt.gz,2.350497,0.9708537,2.833902,4.000901,2.757023
5e010a9a-7f65-434b-ae4c-43948461187c.FPKM.txt.gz,4.409391,2.0214797,2.336283,3.973611,2.063503


In [8]:
max(datExpr)

[1] 15.41908

In [9]:
head(allTraits)

,file_name,cases.0.submitter_id,cases.0.diagnoses.0.morphology,cases.0.diagnoses.0.primary_diagnosis,cases.0.diagnoses.0.tissue_or_organ_of_origin,cases.0.diagnoses.0.tumor_grade,cases.0.diagnoses.0.tumor_stage,cases.0.project.disease_type,cases.0.project.primary_site,cases.0.samples.0.longest_dimension,cases.0.samples.0.portions.0.analytes.0.aliquots.0.submitter_id,cancer.type,Subtype_mRNA,Subtype_DNAmeth,Subtype_protein,Subtype_miRNA,Subtype_CNA,Subtype_Integrative,Subtype_other,Subtype_Selected
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,2d683c1f-3a48-4bb8-84f9-bad6347c008b.FPKM.txt.gz,TCGA-05-4249,8140/3,"Adenocarcinoma, NOS","Lower lobe, lung",not reported,stage ib,Lung Adenocarcinoma,Lung,1.4,TCGA-05-4249-01A-01R-1107-07,LUAD,unknown,unknown,unknown,unknown,unknown,5,unknown,LUAD.5
2,780ab54f-9100-447b-ba9e-75480e7be3d9.FPKM.txt.gz,TCGA-05-4382,8255/3,Adenocarcinoma with mixed subtypes,"Upper lobe, lung",not reported,stage ib,Lung Adenocarcinoma,Lung,0.9,TCGA-05-4382-01A-01R-1206-07,LUAD,unknown,unknown,unknown,unknown,unknown,6,unknown,LUAD.6
3,f8332a4b-0365-4732-a6b8-a3998872e24c.FPKM.txt.gz,TCGA-05-4384,8255/3,Adenocarcinoma with mixed subtypes,"Lower lobe, lung",not reported,stage iiia,Lung Adenocarcinoma,Lung,0.8,TCGA-05-4384-01A-01R-1755-07,LUAD,unknown,low,unknown,unknown,unknown,1,unknown,LUAD.1
4,755f981f-f957-48ec-9779-cf9887e19ab0.FPKM.txt.gz,TCGA-05-4389,8255/3,Adenocarcinoma with mixed subtypes,"Upper lobe, lung",not reported,stage ia,Lung Adenocarcinoma,Lung,1.3,TCGA-05-4389-01A-01R-1206-07,LUAD,unknown,unknown,unknown,unknown,unknown,2,unknown,LUAD.2
5,d0d7fae1-355e-4ada-a469-b69d59c1edfe.FPKM.txt.gz,TCGA-05-4390,8255/3,Adenocarcinoma with mixed subtypes,"Upper lobe, lung",not reported,stage ib,Lung Adenocarcinoma,Lung,1.8,TCGA-05-4390-01A-02R-1755-07,LUAD,unknown,intermediate,unknown,unknown,unknown,2,unknown,LUAD.2
6,97f3c6a3-a3b8-4f59-9242-8845fa1c90a8.FPKM.txt.gz,TCGA-05-4395,8255/3,Adenocarcinoma with mixed subtypes,"Upper lobe, lung",not reported,stage iiib,Lung Adenocarcinoma,Lung,1.0,TCGA-05-4395-01A-01R-1206-07,LUAD,unknown,unknown,unknown,unknown,unknown,3,unknown,LUAD.3


In [11]:
labels <- c('cancer.type', 'Subtype_Selected')
datTraits <- allTraits[,labels]
rownames(datTraits) <- allTraits$file_name #<--CHECK
datTraits <- datTraits[rownames(datExpr),]
head(datTraits)

,cancer.type,Subtype_Selected
,<chr>,<chr>
dc0ce32f-d5aa-489f-ba4b-e01443815121.FPKM.txt.gz,LUAD,LUAD.5
2b33014e-c7ff-41aa-82bf-3121756639ad.FPKM.txt.gz,LUSC,LUSC.primitive
397e302e-393e-4f54-b360-fcffe394271e.FPKM.txt.gz,LUAD,LUAD.1
037b5a94-487e-4f5b-90a3-d696eb4a8a3e.FPKM.txt.gz,LUAD,LUAD.4
5e010a9a-7f65-434b-ae4c-43948461187c.FPKM.txt.gz,LUSC,LUSC.classical
ae59d924-06ec-428d-b470-e78ed92c5e36.FPKM.txt.gz,LUSC,LUSC.classical


In [12]:
table( dimnames(datExpr)[[1]]==datTraits$Subtype_Selected)


FALSE 
  408 

In [13]:
y = datTraits$Subtype_Selected

In [14]:
meanExpressionBySample=apply( datExpr,1,mean, na.rm=T)
NumberMissingBySample=apply( is.na(data.frame(datExpr)),1, sum)

## 1

In [15]:
gsg = goodSamplesGenes(datExpr, verbose = 3);
gsg$allOK

 Flagging genes and samples with too many missing values...
  ..step 1


[1] TRUE

In [16]:
sampleTree = hclust(dist(datExpr), method = "average");
# Plot the sample tree: Open a graphic output window of size 12 by 9 inches
# The user should change the dimensions if the window is too large or too small.
sizeGrWindow(12,9)
svg("samplehier.svg")
par(cex = 0.6);
par(mar = c(0,4,2,0))
plot(sampleTree, main = "Sample clustering to detect outliers", sub="", xlab="", cex.lab = 1.5,
cex.axis = 1.5, cex.main = 2)
dev.off()

png 
  2

In [17]:
# Determine cluster under the line
clust = cutreeStatic(sampleTree, cutHeight = 1e5, minSize = 10)
table(clust)

clust
  1 
408 

In [18]:
# clust 1 contains the samples we want to keep.
keepSamples = (clust==1)
datExpr = datExpr[keepSamples, ]
nGenes = ncol(datExpr)
nSamples = nrow(datExpr)

genes <- dimnames(datExpr)[[2]]
samples <- dimnames(datExpr)[[1]]
datExpr <- as.data.frame(apply(datExpr,2, as.numeric)) #numeric
dimnames(datExpr)[[1]] <- samples

In [19]:
head(datExpr)

,ENSG00000000938,ENSG00000001084,ENSG00000001631,ENSG00000002933,ENSG00000003147,ENSG00000003436,ENSG00000003989,ENSG00000004399,ENSG00000004776,ENSG00000004799,⋯,ENSG00000273284,ENSG00000273302,ENSG00000273338,ENSG00000273340,ENSG00000273350,ENSG00000273369,ENSG00000273373,ENSG00000273399,ENSG00000273449,ENSG00000273474
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
dc0ce32f-d5aa-489f-ba4b-e01443815121.FPKM.txt.gz,2.389567,3.6735564,2.381283,3.494416,3.283922,4.821710,2.8639385,4.918863,1.3895668,3.649615,⋯,0.4646683,0.16349873,0.3785116,0.3785116,0.21412481,0.0976108,0.7311832,0.0000000,0.2750070,0.12432814
2b33014e-c7ff-41aa-82bf-3121756639ad.FPKM.txt.gz,1.819668,6.4294486,3.701549,2.555816,1.438293,1.811471,0.3785116,2.538538,1.1763228,1.722466,⋯,1.8032270,0.48542683,0.1763228,1.6825733,0.56559718,0.4646683,1.0908534,0.0976108,0.5360529,0.05658353
397e302e-393e-4f54-b360-fcffe394271e.FPKM.txt.gz,1.536053,5.3902550,1.992768,3.111031,3.846995,3.404631,4.9731520,2.678072,1.6369146,1.257011,⋯,1.2570106,0.32192809,0.3673711,0.2630344,0.46466827,0.3895668,1.8599695,0.4005379,0.9486008,0.97085365
037b5a94-487e-4f5b-90a3-d696eb4a8a3e.FPKM.txt.gz,2.350497,0.9708537,2.833902,4.000901,2.757023,3.032101,0.9335726,3.655352,0.6040713,2.825786,⋯,1.2570106,0.91073266,0.2868811,0.8479969,0.80735492,1.2448871,1.7697717,1.1634987,1.0840643,0.85598970
5e010a9a-7f65-434b-ae4c-43948461187c.FPKM.txt.gz,4.409391,2.0214797,2.336283,3.973611,2.063503,4.610582,2.5655972,5.360715,5.6892992,4.719731,⋯,0.2868811,0.05658353,0.1375035,0.6040713,0.04264434,0.2868811,0.4005379,0.1375035,0.1763228,0.12432814
ae59d924-06ec-428d-b470-e78ed92c5e36.FPKM.txt.gz,2.821710,6.0205911,3.032101,3.856986,1.056584,1.970854,0.9030383,3.112700,1.0976108,2.687061,⋯,0.7824086,0.26303441,1.0496308,0.4222330,0.79077204,0.3103401,0.8797058,0.0000000,0.3103401,0.07038933


In [20]:
rownames(allTraits) <- allTraits$file_name
datTraits <- allTraits[keepSamples,labels]
rownames(datTraits) <- rownames(allTraits[keepSamples,labels])
datTraits <- datTraits[rownames(datExpr),]
head(datTraits)

,cancer.type,Subtype_Selected
,<chr>,<chr>
dc0ce32f-d5aa-489f-ba4b-e01443815121.FPKM.txt.gz,LUAD,LUAD.5
2b33014e-c7ff-41aa-82bf-3121756639ad.FPKM.txt.gz,LUSC,LUSC.primitive
397e302e-393e-4f54-b360-fcffe394271e.FPKM.txt.gz,LUAD,LUAD.1
037b5a94-487e-4f5b-90a3-d696eb4a8a3e.FPKM.txt.gz,LUAD,LUAD.4
5e010a9a-7f65-434b-ae4c-43948461187c.FPKM.txt.gz,LUSC,LUSC.classical
ae59d924-06ec-428d-b470-e78ed92c5e36.FPKM.txt.gz,LUSC,LUSC.classical


In [21]:
collectGarbage();

In [22]:
# Re-cluster samples
sampleTree2 = hclust(dist(datExpr), method = "average")
fontsize=.5
# Convert traits to a color representation: white means low, red means high, grey means missing entry
traitColors = labels2colors(datTraits);
# Plot the sample dendrogram and the colors underneath.
pdf("samplehiercut.pdf")
plotDendroAndColors(sampleTree2, traitColors,
    groupLabels = labels,
    main = "Sample dendrogram and trait heatmap",
    dendroLabels=FALSE,
    rowText=datTraits,
    rowTextIgnore=labels[[2]]
    )
dev.off()

png 
  2

## 2

In [23]:
# Choose a set of soft-thresholding powers
powers = c(c(1:10), seq(from = 12, to=50, by=2))
# Call the network topology analysis function
sft = pickSoftThreshold(datExpr, powerVector = powers, verbose = 5)
# Plot the results:
sizeGrWindow(9, 5)
pdf("scaletopology.pdf")
par(mfrow = c(1,2));
cex1 = 0.9;
# Scale-free topology fit index as a function of the soft-thresholding power
plot(sft$fitIndices[,1], -sign(sft$fitIndices[,3])*sft$fitIndices[,2],
xlab="Soft Threshold (power)",ylab="Scale Free Topology Model Fit,signed R^2",type="n",
main = paste("Scale independence"));
text(sft$fitIndices[,1], -sign(sft$fitIndices[,3])*sft$fitIndices[,2],
labels=powers,cex=cex1,col="red");
# this line corresponds to using an R^2 cut-off of h
abline(h=0.90,col="red")
# Mean connectivity as a function of the soft-thresholding power
plot(sft$fitIndices[,1], sft$fitIndices[,5],
xlab="Soft Threshold (power)",ylab="Mean Connectivity", type="n",
main = paste("Mean connectivity"))
text(sft$fitIndices[,1], sft$fitIndices[,5], labels=powers, cex=cex1,col="red")
dev.off()

pickSoftThreshold: will use block size 3000.
 pickSoftThreshold: calculating connectivity for given powers...
   ..working on genes 1 through 3000 of 3000
   Power SFT.R.sq  slope truncated.R.sq  mean.k. median.k. max.k.
1      1   0.0142  0.197          0.960 464.0000  4.57e+02 866.00
2      2   0.4760 -0.902          0.932 130.0000  1.14e+02 367.00
3      3   0.6160 -1.240          0.759  50.8000  3.62e+01 190.00
4      4   0.9210 -1.210          0.977  24.6000  1.33e+01 128.00
5      5   0.9440 -1.370          0.983  13.8000  5.48e+00 106.00
6      6   0.9560 -1.400          0.974   8.5500  2.40e+00  89.80
7      7   0.9410 -1.430          0.951   5.7100  1.12e+00  77.80
8      8   0.9600 -1.410          0.961   4.0300  5.60e-01  68.30
9      9   0.9720 -1.390          0.969   2.9700  2.98e-01  60.60
10    10   0.9760 -1.370          0.973   2.2600  1.62e-01  54.30
11    12   0.9650 -1.330          0.957   1.4100  4.93e-02  44.50
12    14   0.9920 -1.280          0.991   0.9520  1.7

png 
  2

In [24]:
net = blockwiseModules(datExpr, power = 30,
TOMType = "unsigned", minModuleSize = 5,
reassignThreshold = 0, mergeCutHeight = 0.2,
numericLabels = TRUE, pamRespectsDendro = FALSE,
saveTOMs = TRUE,
saveTOMFileBase = "wgcna",
verbose = 3)

 Calculating module eigengenes block-wise from all genes
   Flagging genes and samples with too many missing values...
    ..step 1
 ..Working on block 1 .
    TOM calculation: adjacency..
    ..will use 2 parallel threads.
     Fraction of slow calculations: 0.000000
    ..connectivity..
    ..matrix multiplication (system BLAS)..
    ..normalization..
    ..done.
   ..saving TOM for block 1 into file wgcna-block.1.RData
 ....clustering..
 ....detecting modules..
 ....calculating module eigengenes..
 ....checking kME in modules..
 ..merging modules that are too close..
     mergeCloseModules: Merging modules whose distance is less than 0.2
       Calculating new MEs...


In [25]:
# open a graphics window
sizeGrWindow(12, 9)
pdf("netdendograms.pdf")
# Convert labels to colors for plotting
mergedColors = labels2colors(net$colors)
# Plot the dendrogram and the module colors underneath
plotDendroAndColors(net$dendrograms[[1]], mergedColors[net$blockGenes[[1]]],
"Module colors",
dendroLabels = FALSE, hang = 0.03,
addGuide = TRUE, guideHang = 0.05)
dev.off()

png 
  2

In [26]:
moduleLabels = net$colors
moduleColors = labels2colors(net$colors)
MEs = net$MEs;
geneTree = net$dendrograms[[1]];

## 3

In [27]:
datTraits$cancer.type <- match(datTraits$cancer.type, unique(datTraits$cancer.type))
datTraits$Subtype_Selected <- match(datTraits$Subtype_Selected, unique(datTraits$Subtype_Selected))

In [28]:
# Define numbers of genes and samples
nGenes = ncol(datExpr);
nSamples = nrow(datExpr);
# Recalculate MEs with color labels
MEs0 = moduleEigengenes(datExpr, moduleColors)$eigengenes
MEs = orderMEs(MEs0)
moduleTraitCor = cor(MEs, datTraits, use = "p");
moduleTraitPvalue = corPvalueStudent(moduleTraitCor, nSamples);

In [29]:
pdf("moduleTrait.pdf", width=9, height=18)
# Will display correlations and their p-values
textMatrix = paste(signif(moduleTraitCor, 2), "\n(",
signif(moduleTraitPvalue, 1), ")", sep = "");
dim(textMatrix) = dim(moduleTraitCor)
par(mar = c(6, 8.5, 3, 3));
# Display the correlation values within a heatmap plot
labeledHeatmap(Matrix = moduleTraitCor,
xLabels = labels,
yLabels = names(MEs),
ySymbols = names(MEs),
colorLabels = FALSE,
colors = blueWhiteRed(50),
textMatrix = textMatrix,
setStdMargins = FALSE,
cex.text = 0.5,
zlim = c(-1,1),
main = paste("Module-trait relationships"))
dev.off()

png 
  2

In [30]:
# Define variable weight containing the weight column of datTrait
weight = as.data.frame(rownames(datTraits));
names(weight) = "weight"
# names (colors) of the modules
modNames = substring(names(MEs), 3)
geneModuleMembership = as.data.frame(cor(datExpr, MEs, use = "p"));
MMPvalue = as.data.frame(corPvalueStudent(as.matrix(geneModuleMembership), nSamples));
names(geneModuleMembership) = paste("MM", modNames, sep="");
names(MMPvalue) = paste("p.MM", modNames, sep="");
geneTraitSignificance = as.data.frame(cor(datExpr, weight, use = "p"));
GSPvalue = as.data.frame(corPvalueStudent(as.matrix(geneTraitSignificance), nSamples));
names(geneTraitSignificance) = paste("GS.", names(weight), sep="");
names(GSPvalue) = paste("p.GS.", names(weight), sep="");

Warning message in storage.mode(y) <- "double":
“NAs introduced by coercion”


In [31]:
module = "gray"
column = match(module, modNames);
moduleGenes = moduleColors==module;
sizeGrWindow(7, 7);
pdf("modulemembership.pdf")
par(mfrow = c(1,1));
verboseScatterplot(abs(geneModuleMembership[moduleGenes, column]),
    abs(geneTraitSignificance[moduleGenes, 1]),
    xlab = paste("Module Membership in", module, "module"),
    ylab = "Gene significance for primary_site",
    main = paste("Module membership vs. gene significance\n"),
    cex.main = 1.2, cex.lab = 1.2, cex.axis = 1.2, col = module)
dev.off()

ERROR: Error in abs(geneModuleMembership[moduleGenes, column]): non-numeric argument to mathematical function


## 5 

In [ ]:
# Re-cluster samples
sampleTree3 = hclust(dist(MEs0), method = "average")
# Convert traits to a color representation: white means low, red means high, grey means missing entry
traitColors = labels2colors(datTraits);
# Plot the sample dendrogram and the colors underneath.
pdf("nethier.pdf", width=10)
# Set the plot margin: bottom, left, top & right
par(mar = c(5, 4, 1.4, 0.2) + 0.1,
    xpd = NA) # allow content to go into outer margin 
plotDendroAndColors(sampleTree3, traitColors,
groupLabels = labels,
main = "Network dendrogram and trait heatmap",
dendroLabels=FALSE,
#rowText=allTraits[keepSamples,labels],
#rowTextIgnore=labels[[2]],
addGuide=TRUE)
dev.off()

In [ ]:
rownames(allTraits)=allTraits$file_name
unique(allTraits[rownames(datTraits),'Subtype_Selected'])

In [ ]:
svg("nethier_legend.svg", width=10)
add_legend("topleft", legend = unique(allTraits[rownames(datTraits),'SMTS']), pch = 1, pt.cex = 1, cex = 1.5, bty = 'n', ncol=2,
       inset = c(0.1, 0.), # place outside
       title = "SMTS", 
       col = traitColors)
dev.off()

In [ ]:
collectGarbage()

In [ ]:
clust = cutreeStatic(sampleTree3, cutHeight = 0.4, minSize = 10)
table(clust)

In [ ]:
# Re-cluster samples after cut
sampleTree4 = hclust(dist(MEs0[(clust==1),]), method = "average")
# Convert traits to a color representation: white means low, red means high, grey means missing entry
traitColors = labels2colors(datTraits[(clust==1),]);
# Plot the sample dendrogram and the colors underneath.
svg("nethiercut.svg")
plotDendroAndColors(sampleTree4, traitColors,
groupLabels = labels,
main = "Network dendrogram and trait heatmap",
dendroLabels=FALSE)
dev.off()

In [ ]:
# Calculate topological overlap anew: this could be done more efficiently by saving the TOM
# calculated during module detection, but let us do it again here.
dissTOM = 1-TOMsimilarityFromExpr(datExpr, power = 6);
# Transform dissTOM with a power to make moderately strong connections more visible in the heatmap
plotTOM = dissTOM^7;
# Set diagonal to NA for a nicer plot
diag(plotTOM) = NA;
# Call the plot function
sizeGrWindow(9,9)
svg("networkmap.svg")
TOMplot(plotTOM, geneTree, moduleColors, main = "Network heatmap plot, all genes")
dev.off()

In [ ]:
#selct genes to be faster
nSelect = nGenes*0.7
# For reproducibility, we set the random seed
set.seed(10);
select = sample(nGenes, size = nSelect);
selectTOM = dissTOM[select, select];
# There’s no simple way of restricting a clustering tree to a subset of genes, so we must re-cluster.
selectTree = hclust(as.dist(selectTOM), method = "average")
selectColors = moduleColors[select];
# Open a graphical window
sizeGrWindow(9,9)
svg("networkmap.svg")
# Taking the dissimilarity to a power, say 10, makes the plot more informative by effectively changing
# the color palette; setting the diagonal to NA also improves the clarity of the plot
plotDiss = selectTOM^7;
diag(plotDiss) = NA;
TOMplot(plotDiss, selectTree, selectColors, main = "Network heatmap plot, selected genes")
dev.off()

In [ ]:
# Recalculate module eigengenes
MEs = moduleEigengenes(datExpr, moduleColors)$eigengenes
# Isolate weight from the clinical traits
weight = as.data.frame(datTraits$SMTS);
names(weight) = "SMTS"
# Add the weight to existing module eigengenes
MET = orderMEs(cbind(MEs, weight))
# Plot the relationships among the eigengenes and the trait
sizeGrWindow(5,7.5);
svg("eigenHeat.svg")
par(cex = 0.9)
plotEigengeneNetworks(MET, "", marDendro = c(0,4,1,2), marHeatmap = c(3,4,1,2), cex.lab = 0.8, xLabelsAngle = 90)
dev.off()

In [ ]:
# Plot the dendrogram
sizeGrWindow(6,6);
svg("eigendendogram.svg")
par(cex = 1.0)
plotEigengeneNetworks(MET, "Eigengene dendrogram", marDendro = c(0,4,2,0),
plotHeatmaps = FALSE)
# Plot the heatmap matrix (note: this plot will overwrite the dendrogram plot)
par(cex = 1.0)
plotEigengeneNetworks(MET, "Eigengene adjacency heatmap", marHeatmap = c(3,4,2,2),
plotDendrograms = FALSE, xLabelsAngle = 90)
dev.off()

Values of *k* must be read from *clustersizes.txt file*

In [ ]:
write.csv(cutree(sampleTree3, k=2), "wgcna/wgcna_level_2_labels.csv")
write.csv(cutree(sampleTree3, k=8), "wgcna/wgcna_level_1_labels.csv")
write.csv(cutree(sampleTree3, k=58), "wgcna/wgcna_level_0_labels.csv")

In [32]:
write.csv(geneModuleMembership, "wgcna/wgcna_level_0_word-dist.csv")

In [33]:
write.csv(MEs, "wgcna/wgcna_level_0_topic-dist.csv")